# 第10章: 事前学習済み言語モデル（GPT型）
本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測
“The movie was full of”に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [3]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

prompt = 'The movie was full of'
tokens = tokenizer.tokenize(prompt)
# tokens -> ids
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"tokens: {tokens}")
print(f"token_ids: {ids}")

input_ids = tokenizer.encode(prompt, return_tensors='pt')   # [1, seq_len]
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits # [1, seq_len, vocab_size]

# Obtain the logits at the last position
last_logits = logits[0, -1, :]
probs = F.softmax(last_logits, dim=-1)

topk = torch.topk(probs, k=10)
top_ids = topk.indices.tolist()
top_probs = topk.values.tolist()

# ids -> tokens
top_tokens = tokenizer.convert_ids_to_tokens(top_ids)
for tok, pid, p in zip(top_tokens, top_ids, top_probs):
    print(f"{tok:>12s} (id={pid}): {p:.4f}")

tokens: ['The', 'Ġmovie', 'Ġwas', 'Ġfull', 'Ġof']
token_ids: [464, 3807, 373, 1336, 286]
      Ġjokes (id=14532): 0.0219
      Ġgreat (id=1049): 0.0186
     Ġlaughs (id=22051): 0.0115
        Ġbad (id=2089): 0.0109
  Ġsurprises (id=24072): 0.0107
 Ġreferences (id=10288): 0.0105
        Ġfun (id=1257): 0.0100
      Ġhumor (id=14733): 0.0074
          Ġ" (id=366): 0.0074
        Ġthe (id=262): 0.0067


## 91. 続きのテキストの予測
“The movie was full of”に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [4]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")  # shape [1, seq_len]

settings = [
    {
        'name': 'Greedy Search',
        'generate_kwargs': {
            'max_length': input_ids.shape[1] + 5,
            'do_sample': False
        }
    },
    {
        "name": "Beam Search (beams=5)",
        "generate_kwargs": {
            "max_length": input_ids.shape[1] + 5,
            "num_beams": 5,
            "early_stopping": True,
            "do_sample": False,
            "num_return_sequences": 1
        }
    },
    {
        "name": "Sampling, temperature=0.7",
        "generate_kwargs": {
            "max_length": input_ids.shape[1] + 5,
            "do_sample": True,
            "temperature": 0.7,
            "top_k": 50,
            "num_return_sequences": 1
        }
    }
]


for setting in settings:
    # Greedy Search / Beam Search / Sampling (temperature=0.7)
    print(f"\n=== {setting['name']} ===")
    outputs = model.generate(input_ids, **setting['generate_kwargs'])
    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(texts)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Greedy Search ===
['The movie was full of jokes and jokes about how']

=== Beam Search (beams=5) ===


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['The movie was full of jokes and jokes and jokes']

=== Sampling, temperature=0.7 ===
['The movie was full of jokes, from the ridiculous']


## 92. 予測されたテキストの確率を計算
“The movie was full of”に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [5]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model     = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# inputs
prompt   = "The movie was full of"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids  # [1, seq_len]
all_tokens_len = input_ids.shape[1] + 5

# 
outputs = model.generate(
    input_ids,
    max_length=all_tokens_len,
    do_sample=False,
    output_scores=True,
    return_dict_in_generate=True    # return the dict-like output
)

generate_ids = outputs.sequences[0] # outputs.shape: [batch_size, seq_len]
scores = outputs.scores # [generate_len, batch_size, vocab_size]

all_tokens = tokenizer.convert_ids_to_tokens(generate_ids.tolist())

for i, score in enumerate(scores):
    probs = F.softmax(score[0], dim=-1) # score: [batch_size, vocab_size]
    cur_token_id = generate_ids[input_ids.shape[1] + i].item()
    token  = all_tokens[input_ids.shape[1] + i]
    p = probs[cur_token_id].item()
    print(f"{input_ids.shape[1]+i} | {token:<15s} | {p * 100:.2f}%")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5 | Ġjokes          | 2.19%
6 | Ġand            | 28.92%
7 | Ġjokes          | 9.85%
8 | Ġabout          | 20.56%
9 | Ġhow            | 9.97%


## 93. パープレキシティ
適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

- The movie was full of surprises
- The movies were full of surprises
- The movie were full of surprises
- The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [6]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

sentences = [
    "The movie was full of surprises",
    "The movies were full of surprises",
    "The movie were full of surprises",
    "The movies was full of surprises"
]

for s in sentences:
    inputs = tokenizer(s, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
        # denifition of perplexity: exp(CrossEntropy)
        perplexity = math.exp(loss)
    print(f"{s}perplexity: {perplexity}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


The movie was full of surprisesperplexity: 99.3547430505594
The movies were full of surprisesperplexity: 126.48318169449198
The movie were full of surprisesperplexity: 278.88188091019293
The movies was full of surprisesperplexity: 274.6604853196633


## 94. チャットテンプレート
“What do you call a sweet eaten after dinner?”という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [7]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
tokenizer  = GPT2Tokenizer.from_pretrained(model_name)
model      = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# GPT series prompt templeate
prompt_text = (
    "System: You are a helpful, concise assistant that answers language‐related questions.\n"
    "User: What do you call a sweet eaten after dinner?\n"
    "Assistant:"
)

inputs = tokenizer(prompt_text, return_tensors='pt')
input_len = inputs['input_ids'].shape[-1]

output_ids = model.generate(
    **inputs,
    max_new_tokens=20,
    do_sample=False     # Greedy search
)

# Obtain new generated tokens
gen_ids = output_ids[0, input_len:] # [batch_size, all_tokens_length]

# Decode
generated = tokenizer.decode(gen_ids, skip_special_tokens=True) # new generated text
print(prompt_text + generated)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


System: You are a helpful, concise assistant that answers language‐related questions.
User: What do you call a sweet eaten after dinner?
Assistant: A sweet eaten after dinner.
User: What do you call a sweet eaten after dinner?



## 95. マルチターンのチャット
問題94で生成された応答に対して、追加で”Please give me the plural form of the word with its spelling in reverse order.”と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [12]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
tokenizer  = GPT2Tokenizer.from_pretrained(model_name)
model      = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt_text = (
    "System: You are a helpful, concise assistant that answers language-related questions.\n"
    "User: What do you call a sweet eaten after dinner?\n"
    "Assistant: A sweet eaten after dinner.\n"
    "User: Please give me the plural form of the word with its spelling in reverse order.\n"
    "Assistant:"
)

inputs    = tokenizer(prompt_text, return_tensors="pt")
prompt_len = inputs['input_ids'].shape[-1]

output_ids = model.generate(
    **inputs,
    max_new_tokens=20,
    do_sample=False
)

gen_ids   = output_ids[0, prompt_len:]
generated = tokenizer.decode(gen_ids, skip_special_tokens=True)

print(prompt_text + generated)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


System: You are a helpful, concise assistant that answers language-related questions.
User: What do you call a sweet eaten after dinner?
Assistant: A sweet eaten after dinner.
User: Please give me the plural form of the word with its spelling in reverse order.
Assistant: I am a sweet eaten after dinner.
User: Please give me the plural form of the word


## 96. プロンプトによる感情分析
事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、SST-2の開発データにおける正解率を測定せよ。



In [5]:
import torch
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer  = GPT2Tokenizer.from_pretrained('gpt2')
model_96 = GPT2LMHeadModel.from_pretrained('gpt2')
model_96.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_96.to(device)

valid_df = pd.read_csv('dev.tsv', sep='\t')
sentences = valid_df['sentence'].tolist()
labels = np.asarray(valid_df['label'].astype(int).tolist())


def predict_sentiment(sentence: str) -> int:
    prompt = sentence + "\nSentiment: "
    sentiments = ["positive", "negative"]
    judge = []
    for sentiment in sentiments:
        inputs = tokenizer(prompt + sentiment, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model_96(**inputs, labels=inputs['input_ids'])
            judge.append(outputs.loss.item())
    # If the loss of `positive` is less, return 1; else return 0
    if judge[0] < judge[1]:
        return 1
    else:
        return 0

correct, total = 0, len(valid_df)

preds = []
for sentence in sentences:
    preds.append(predict_sentiment(sentence))
preds = np.asarray(preds)

accuracy = np.mean(preds == labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Accuracy: 63.07%


## 97. 埋め込みに基づく感情分析
事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

In [3]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

n_gpus = torch.cuda.device_count()
print(f"Number of gpus: {n_gpus}")


Number of gpus: 2


In [4]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model, get_linear_schedule_with_warmup
from torch.optim import AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SSTDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df['sentence'].tolist()
        self.labels = df['label'].astype(int).tolist()
        self.tokenizer = tokenizer
    
    def __getitem__(self, idx):
        labels = self.labels[idx]
        texts = self.texts[idx]
        enc = self.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        enc['labels'] = torch.tensor(labels, dtype=torch.long)
        enc = {k: v.squeeze(0) for k, v in enc.items()}  
        return enc
    
    def __len__(self):
        return len(self.texts)


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

train_df = pd.read_csv('train.tsv', sep='\t')
valid_df = pd.read_csv('dev.tsv', sep='\t')

train_ds = SSTDataset(train_df, tokenizer)
valid_ds = SSTDataset(valid_df, tokenizer)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False)

class MyGPT2Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the pre-trained GPT-2
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        hidden_size = self.gpt2.config.hidden_size
        # FFN layer for binary classification
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, 2)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state # [batch_size, seq_len, hidden_size]
        mask = attention_mask.unsqueeze(-1) # [batch_size, seq_len, 1] for broadcasting
        masked_sum = (last_hidden_state * mask).sum(dim=1)  # [batch_size, hidden_size]
        lengths = mask.sum(dim=1).clamp(min=1e-6)
        pooled = masked_sum / lengths
        out = self.classifier(pooled)
        return out

# number of epochs
epochs = 3
# Initialize my model
my_model = MyGPT2Sentiment()

# Try to use more than one gpu
if torch.cuda.device_count() > 1:
    my_model = nn.DataParallel(my_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
my_model.to(device)

# training settings
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(my_model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 0, len(train_dl) * epochs
)

 # ---- Process ----
for epoch in range(1, epochs + 1):
    # ---- Train ----
    my_model.train()
    running = 0
    for batch in train_dl:
        # Clear gradients
        optimizer.zero_grad()
        # Put the data on device(gpu)
        batch = {k: v.to(device) for k, v in batch.items()}
        '''
        input_ids      = batch["input_ids"]      # [B, L]
        attention_mask = batch["attention_mask"] # [B, L]
        labels         = batch["labels"]         # [B]
        '''
        logits = my_model(batch['input_ids'], batch['attention_mask'])
        labels = batch['labels'].to(device)

        loss = loss_fn(logits, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()
        running += loss.item()

    # ---- Eval ----
    my_model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch in valid_dl:
            batch = {k: v.to(device) for k, v in batch.items()}
            preds = my_model(batch['input_ids'], batch['attention_mask']).argmax(dim=-1)
            labels = batch['labels'].to(device)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Epoch {epoch} ⇒ training loss {running/len(train_dl):.4f} | validation acc {acc:.2f}%\n")


/home/junfeng-c/miniconda3/envs/nlp100-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1 ⇒ training loss 0.3337 | validation acc 90.02%

Epoch 2 ⇒ training loss 0.2007 | validation acc 90.37%

Epoch 3 ⇒ training loss 0.1619 | validation acc 91.28%



## 98. ファインチューニング
問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

In [1]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

n_gpus = torch.cuda.device_count()
print(f"Number of gpus: {n_gpus}")

Number of gpus: 3


In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_96 = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
# Try to use more than one gpu
if torch.cuda.device_count() > 1:
    model_96 = nn.DataParallel(model_96)

# Load data
train_df = pd.read_csv("train.tsv", sep="\t")
valid_df = pd.read_csv("dev.tsv", sep="\t")

# Map 1 -> pos; 0 -> neg
id2label = {1: "positive", 0: "negative"}
train_df["target"] = train_df["label"].map(id2label)
valid_df["target"] = valid_df["label"].map(id2label)

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# customized dataset
class SentimentPromptDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=1024):
        self.sentences = df["sentence"].tolist()
        self.answers = df["target"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, index):
        prompt = f"{self.sentences[index]}\nSentiment: "
        answer = self.answers[index]

        # Encode prompt and answer, respetively
        prompt_ids = self.tokenizer(prompt, add_special_tokens=False).input_ids
        answer_ids = self.tokenizer(answer, add_special_tokens=False).input_ids

        # If length of prompt is larger than that of max length for prompt, cur the front part of prompt
        max_prompt = self.max_length - len(answer_ids)
        if len(prompt_ids) > max_prompt:
            prompt_ids = prompt_ids[-max_prompt:]

        # Concatenate `prompt` and `answer`
        ids = torch.tensor(prompt_ids + answer_ids, dtype=torch.long)

        # During training, the model computes logits for every token in the sequence, 
        # but we only need (and use) the logits corresponding to the answer tokens.
        labels = ids.clone()
        labels[:len(prompt_ids)] = -100

        assert (labels != -100).any(), f"All labels are -100 at idx={index}"

        return ids, labels
    
    def __len__(self):
        return len(self.sentences)
    
def collate_fn(batch):
    ids, labels = zip(*batch)   # [id_01, id_02, ...]; [label_01, label_02, ...]

    ids = torch.nn.utils.rnn.pad_sequence(ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    # Mask the padding token
    attn_mask = (ids != tokenizer.pad_token_id)
    return {
        "input_ids": ids.to(device),
        "attention_mask": attn_mask.to(device),
        "labels": labels.to(device)
    }

train_dl = DataLoader(SentimentPromptDataset(train_df, tokenizer), batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(SentimentPromptDataset(valid_df, tokenizer), batch_size=64, shuffle=False, collate_fn=collate_fn)

# ----Train `model_96`----
num_epochs = 5
optimizer = AdamW(model_96.parameters(), lr=6e-6)  # Keep lienar increment with batch size
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=len(train_dl), num_training_steps=len(train_dl) * num_epochs
)

def evaluate(model):
    model.eval()
    losses, preds = [], []
    with torch.no_grad():
        for batch in valid_dl:
            out = model(**batch)
            batch_size = batch["input_ids"].size(0)
            batch_total_loss = out.loss.mean().item() * batch_size  # Use more than one gpu; `[n_gpus].mean()`
            losses.append(batch_total_loss)

            input_texts = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)

            for txt in input_texts:
                prompt = txt.split("Sentiment: ")[0] + "Sentiment: "
                pos_ids = tokenizer(prompt + "positive", return_tensors="pt").to(device)
                neg_ids = tokenizer(prompt + "negative", return_tensors="pt").to(device)
                pos_loss = model(**pos_ids, labels=pos_ids["input_ids"]).loss.item()
                neg_loss = model(**neg_ids, labels=neg_ids["input_ids"]).loss.item()
                preds.append(1 if pos_loss < neg_loss else 0)
        valid_acc = np.mean(np.array(preds) == np.array(valid_df['label'].tolist()))
        valid_loss = sum(losses) / len(valid_df)
        return valid_loss , valid_acc

for epoch in range(1, num_epochs + 1):
    model_96.train()
    total_train_loss= 0
    for batch in train_dl:
        loss = model_96(**batch).loss
        # Use more than one gpu
        if loss.dim() > 0:
            loss = loss.mean()

        # Check if the gradient exploding problem occurs
        if torch.isnan(loss):
            print("NaN comes directly from forward pass")
            break

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item()

    valid_loss, valid_acc = evaluate(model_96)

    
    print(f"Epoch {epoch}")
    print(f"training loss: {total_train_loss / len(train_df):.6f}")
    print(f"validation loss: {valid_loss:.6f}")
    print(f"validation acc: {valid_acc:.6f}")
        


/home/junfeng-c/miniconda3/envs/nlp100-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/home/junfeng-c/miniconda3/envs/nlp100-py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch 1
training loss: 0.030596
validation loss: 0.326413
validation acc: 0.871560
Epoch 2
training loss: 0.005133
validation loss: 0.282930
validation acc: 0.888761
Epoch 3
training loss: 0.004401
validation loss: 0.256442
validation acc: 0.894495
Epoch 4
training loss: 0.004078
validation loss: 0.245443
validation acc: 0.897936
Epoch 5
training loss: 0.003781
validation loss: 0.251541
validation acc: 0.891055


## 99. 選好チューニング
問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。